In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments, EvalPrediction,AutoModelForSequenceClassification,AutoModelForCausalLM
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model,  TaskType
from datasets import  load_dataset ,Dataset,DatasetDict,load_metric
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import sentencepiece
from transformers import T5Tokenizer, T5ForConditionalGeneration


/Users/saahil/Desktop/Coding_Projects/LLMS/LoraPeft-BioMRc/env2/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/saahil/Desktop/Coding_Projects/LLMS/LoraPeft-BioMRc/env2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import wandb


# Initialize a new run
wandb.init(project='loramedical')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/Users/saahil/Desktop/Coding_Projects/LLMS/LoraPeft-BioMRc/env2/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
wandb: Currently logged in as: saahilkatariads (saahilkatariads-MCKV Institute of Engineering). Use `wandb login --relogin` to force relogin


In [4]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")



In [5]:
dataset = pd.read_csv('train.csv')
dataset = dataset.drop('qtype', axis=1)
dataset = dataset.rename(columns={'Question': 'question', 'Answer': 'answer'})

In [6]:
df_full_train, df_test = train_test_split(dataset, test_size=0.2, random_state=56)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=56)

In [7]:
df_train = df_train.reset_index(drop=True)
df_val = df_train.reset_index(drop=True)
df_test = df_train.reset_index(drop=True)
train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)

In [8]:
health_dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

In [9]:
import sentencepiece
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map=device)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Users/saahil/Desktop/Coding_Projects/LLMS/LoraPeft-BioMRc/env2/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
prefix = "Assuming you are working as Doctor. Please answer this question: "



def preprocess_function(examples):
   inputs = [prefix + doc for doc in examples["question"]]
   model_inputs = tokenizer(inputs, max_length=256, truncation=True)
   labels = tokenizer(text_target=examples["answer"], 
                      max_length=256,         
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [11]:
tokenized_dataset = health_dataset_dict.map(preprocess_function, batched=True)

Map: 100%|██████████| 9843/9843 [00:05<00:00, 1870.54 examples/s]


In [12]:
peft_config = LoraConfig(
    r=32,  # Rank
    lora_alpha=16,  # Scaling
    lora_dropout=0.01,  # Dropout probability
    bias="none",  # Bias term setting
    target_modules=["q",'v'],
    task_type=TaskType.SEQ_2_SEQ_LM
        # Apply LoRA to the query and value projection layers
     
)

In [13]:
model = get_peft_model(model, peft_config)

In [15]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',           # output directory
    evaluation_strategy='epoch',      # evaluation is done at the end of each epoch
    learning_rate=1e-4,               # learning rate
    per_device_train_batch_size=8,    # batch size for training
    per_device_eval_batch_size=16,    # batch size for evaluation
    weight_decay=0.01,                # strength of weight decay
    save_total_limit=3,               # limits the total amount of checkpoints. Deletes older checkpoints.
    num_train_epochs=15,               # number of training epochs
    predict_with_generate=True,       # generate predictions for evaluation
    report_to='wandb'
)

/Users/saahil/Desktop/Coding_Projects/LLMS/LoraPeft-BioMRc/env2/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
    
)

In [17]:

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
  3%|▎         | 500/18465 [2:04:01<120:43:12, 24.19s/it]  

{'loss': 2.273, 'grad_norm': 0.1970219910144806, 'learning_rate': 9.729217438396968e-05, 'epoch': 0.41}


/Users/saahil/Desktop/Coding_Projects/LLMS/LoraPeft-BioMRc/env2/lib/python3.9/site-packages/peft/utils/other.py:619: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: aefcc6c7-8877-4baa-be43-59d89a4fdb16)') - silently ignoring the lookup for the file config.json in google/flan-t5-large.
  warnings.warn(
/Users/saahil/Desktop/Coding_Projects/LLMS/LoraPeft-BioMRc/env2/lib/python3.9/site-packages/peft/utils/save_and_load.py:218: UserWarning: Could not find a config file in google/flan-t5-large - will assume that the vocabulary was not modified.
  warnings.warn(
  5%|▌         | 1000/18465 [4:41:40<8:08:24,  1.68s/it]   

{'loss': 2.024, 'grad_norm': 0.2271500676870346, 'learning_rate': 9.458434876793935e-05, 'epoch': 0.81}


  7%|▋         | 1231/18465 [8:10:47<6:13:50,  1.30s/it]

...

 97%|█████████▋| 18000/18465 [23:56:41<30:42,  3.96s/it]

{'loss': 1.7034, 'grad_norm': 0.2590237259864807, 'learning_rate': 2.5182778229082047e-06, 'epoch': 14.62}


                                                        
100%|██████████| 18465/18465 [24:31:43<00:00,  4.78s/it]

{'eval_loss': 1.5114909410476685, 'eval_runtime': 925.7911, 'eval_samples_per_second': 10.632, 'eval_steps_per_second': 0.665, 'epoch': 15.0}
{'train_runtime': 88303.587, 'train_samples_per_second': 1.672, 'train_steps_per_second': 0.209, 'train_loss': 1.78713118106151, 'epoch': 15.0}


TrainOutput(global_step=18465, training_loss=1.78713118106151, metrics={'train_runtime': 88303.587, 'train_samples_per_second': 1.672, 'train_steps_per_second': 0.209, 'total_flos': 2.59039270075392e+16, 'train_loss': 1.78713118106151, 'epoch': 15.0})

In [ ]:
final_results = trainer.evaluate()



In [21]:
print(final_results)

{'eval_runtime': 70.4313, 'eval_samples_per_second': 139.753, 'eval_steps_per_second': 8.746, 'epoch': 5.0}


In [18]:
def generate_answer(question, model, tokenizer, max_length=512):
    input_text = "Assuming you are working as Doctor. Please answer this question: " + question
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(
        input_ids=inputs["input_ids"], 
        max_length=256, 
        num_beams=5, 
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer



In [33]:
# Example question 1
question = "What are the treatments for spinocerebellar ataxia type 36"

# Generate the answer
answer = generate_answer(question, model, tokenizer)
print("Question:", question)
print("Answer:", answer)

Question: What are the treatments for spinocerebellar ataxia type 36
Answer: These resources address the diagnosis or management of spinocerebellar ataxia type 36: - Gene Review: Gene Review: Spinocerebellar Ataxia Type 36 - Genetic Testing Registry: Spinocerebellar ataxia type 36 These resources from MedlinePlus offer information about the diagnosis and management of various health conditions: - Diagnostic Tests - Drug Therapy - Surgery and Rehabilitation - Genetic Counseling - Palliative Care


In [21]:
# Example question 1
question = "Is Fuchs endothelial dystrophy inherited ?"

# Generate the answer
answer = generate_answer(question, model, tokenizer)
print("Question:", question)
print("Answer:", answer)

Question: Is Fuchs endothelial dystrophy inherited ?
Answer: This condition is inherited in an autosomal recessive pattern, which means both copies of the gene in each cell have mutations. The parents of an individual with an autosomal recessive condition each carry one copy of the mutated gene, but they typically do not show signs and symptoms of the condition.


In [20]:
train_dataset[4]

{'question': 'Is Fuchs endothelial dystrophy inherited ?',
 'answer': 'In some cases, Fuchs endothelial dystrophy appears to be inherited in an autosomal dominant pattern, which means one copy of the altered gene in each cell is sufficient to cause the disorder. When this condition is caused by a mutation in the COL8A2 gene, it is inherited in an autosomal dominant pattern. In addition, an autosomal dominant inheritance pattern is apparent in some situations in which the condition is caused by alterations in an unknown gene.  In many families, the inheritance pattern is unknown.  Some cases result from new mutations in a gene and occur in people with no history of the disorder in their family.'}

In [22]:
model.save_pretrained('./biomrcmodel')

In [23]:
tokenizer.save_pretrained('./biotokenizer')

('./biotokenizer/tokenizer_config.json',
 './biotokenizer/special_tokens_map.json',
 './biotokenizer/spiece.model',
 './biotokenizer/added_tokens.json')